In [ ]:
# @title
### EP2 MAC0417 / MAC5768 ##################################################################
# AO PREENCHER ESSE CABEÇALHO COM O MEU NOME E O MEU NÚMERO USP,#
# DECLARO QUE SOU O ÚNICO AUTOR E RESPONSÁVEL PELA RESOLUÇÃO #
# DESTE EP. # # TODAS AS PARTES FORAM DESENVOLVIDAS E IMPLEMENTADAS POR MIM, #
# SEGUINDO AS INSTRUÇÕES E QUE PORTANTO, NÃO CONSTITUEM # # DESONESTIDADE ACADÊMICA OU PLÁGIO. #
## # DECLARO TAMBÉM, QUE SOU RESPONSÁVEL POR TODAS AS CÓPIAS # # DESSE PROGRAMA, E QUE EU NÃO DISTRIBUI OU FACILITEI A #
# SUA DISTRIBUIÇÃO. ESTOU CIENTE QUE OS CASOS DE PLÁGIO E # # DESONESTIDADE ACADÊMICA SERÃO TRATADOS SEGUNDO OS CRITÉRIOS #
# DEFINIDOS NO CÓDIGO DE ÉTICA DA USP. # ## # ENTENDO QUE JUPYTER NOTEBOOKS SEM ASSINATURA NÃO SERÃO #
# CORRIGIDOS E, AINDA ASSIM, PODERÃO SER PUNIDOS POR # # DESONESTIDADE ACADÊMICA. #
## ##
# Nome: Gabriela Victor #
# NUSP: 11795381 #
# Turma: MAC0417 145    #
# Prof.: Ronaldo Fumio Hashimoto #
##################################################################

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import matplotlib.image as mpimg
import pandas as pd
import os
import cv2

In [2]:

!git clone https://github.com/gabriela-vic/EP2-MAC0417


Cloning into 'EP2-MAC0417'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 232 (delta 2), reused 119 (delta 1), pack-reused 112
Receiving objects: 100% (232/232), 98.57 MiB | 17.25 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Updating files: 100% (219/219), done.


In [4]:
os.chdir('EP2-MAC0417')

In [11]:
os.listdir()

['augmentationDataSet',
 'MAC0417_EP2_PARTE1.ipynb',
 'originalGrayDataSet',
 '.git']

In [ ]:

# Função de equalização de histograma
def equalize_histogram(img):

    hist, bins = np.histogram(img.flatten(), 256, [0,256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * hist.max()/ cdf.max()


    cdf_m = np.ma.masked_equal(cdf, 0)
    cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())
    cdf = np.ma.filled(cdf_m, 0).astype('uint8')


    img_equalized = cdf[img]
    return img_equalized

# Função para calcular o histograma médio
def calculate_average_histogram(image_files, dir_path):
    histograms = []
    for image_file in image_files:
        img_path = os.path.join(dir_path, image_file)
        img = plt.imread(img_path)
        if img.ndim == 3:
            img = np.dot(img[..., :3], [0.2989, 0.5870, 0.1140])  # Converter para escala de cinza

        # Calcular histograma
        hist, bins = np.histogram(img.flatten(), 256, [0,256])
        histograms.append(hist)

    # Calcular histograma médio
    avg_hist = np.mean(histograms, axis=0)
    return avg_hist

# Caminhos dos diretórios e arquivos
original_gray_dir = "/content/EP2-MAC0417/grayscaleDataSet"
augmented_dir = "/content/EP2-MAC0417/augmentationDataSet"
normalized_dir = "/content/EP2-MAC0417/normalizedDataSet"

original_gray_csv_path = os.path.join(original_gray_dir, "grayscaleDataSet.csv")
augmented_csv_path = os.path.join(augmented_dir, "augmentationDataSet.csv")

# Carregar CSVs
df_original = pd.read_csv(original_gray_csv_path)
df_augmented = pd.read_csv(augmented_csv_path)

# Criar diretório normalizedDataset se não existir
if not os.path.exists(normalized_dir):
    os.makedirs(normalized_dir)

# Normalizar imagens do augmentedDataset
for image_file in df_augmented['filename']:
    img_path = os.path.join(augmented_dir, image_file)
    img = plt.imread(img_path)
    if img.ndim == 3:
        img = np.dot(img[..., :3], [0.2989, 0.5870, 0.1140])  # Converter para escala de cinza

    # Aplicar equalização de histograma
    norm_img = equalize_histogram(img)

    # Salvar imagem normalizada
    norm_img_path = os.path.join(normalized_dir, image_file)
    plt.imsave(norm_img_path, norm_img, cmap='gray')

# Calcular histogramas médios
hist_original = calculate_average_histogram(df_original['filename'], original_gray_dir)
hist_augmented = calculate_average_histogram(df_augmented['filename'], augmented_dir)
hist_normalized = calculate_average_histogram(df_augmented['filename'], normalized_dir)

# Visualizar uma amostra de imagens antes e depois da normalização
def plot_image_comparison(original_img, normalized_img, title_original='Original', title_normalized='Normalized'):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(original_img, cmap='gray')
    plt.title(title_original)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(normalized_img, cmap='gray')
    plt.title(title_normalized)
    plt.axis('off')

    plt.tight_layout()
    plt.show()

sample_images = df_augmented['filename'].sample(3)
for image_file in sample_images:
    original_img_path = os.path.join(augmented_dir, image_file)
    normalized_img_path = os.path.join(normalized_dir, image_file)

    original_img = plt.imread(original_img_path)
    if original_img.ndim == 3:
        original_img = np.dot(original_img[..., :3], [0.2989, 0.5870, 0.1140])  # Converter para escala de cinza

    normalized_img = plt.imread(normalized_img_path)
    if normalized_img.ndim == 3:
        normalized_img = np.dot(normalized_img[..., :3], [0.2989, 0.5870, 0.1140])  # Converter para escala de cinza

    plot_image_comparison(original_img, normalized_img)

# Plotar histogramas médios
plt.figure(figsize=(10, 5))
plt.plot(hist_original, label='Original Gray')
plt.plot(hist_augmented, label='Augmented')
plt.plot(hist_normalized, label='Normalized')
plt.title('Histogramas Médios')
plt.xlabel('Intensidade')
plt.ylabel('Frequência')
plt.legend()
plt.show()

